In [1]:
import pandas as pd
import numpy as np
import nltk
import chardet
from flair.data import Sentence
from flair.models import SequenceTagger
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm_notebook, tqdm

# spooky

In [18]:
path = './dataset/Spooky Author Identification/split/'

df_train = pd.read_csv(path+'train.csv', delimiter=',', index_col= 0, encoding='utf-8') 
df_test = pd.read_csv(path+'test.csv', delimiter=',', index_col= 0, encoding='utf-8') 
df_dev = pd.read_csv(path+'dev.csv', delimiter=',', index_col= 0, encoding='utf-8') 
df_total = pd.read_csv(path+'total.csv', delimiter=',', index_col= 0, encoding='utf-8') 

df_train

,id,text,author
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
5,id22965,"A youth passed in solitude, my best years spen...",MWS
8,id19322,I knew that you could not say to yourself 'ste...,EAP
...,...,...,...
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP


In [20]:
id_list = df_total['id'].values.tolist()
text_list = df_total['text'].values.tolist()
author_list = df_total['author'].values.tolist()

In [4]:
tagger = SequenceTagger.load('pos')
flair_pos_sentences = []
for text in tqdm_notebook(text_list):
    sentence = Sentence(text)    
    tagger.predict(sentence)
    tagged_sentence = sentence.to_tagged_string()
    flair_pos_sentences.append(tagged_sentence)

2020-10-22 02:01:51,642 loading file /home/joey/.flair/models/en-pos-ontonotes-v0.5.pt


<ipython-input-4-5ef82793719b>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm_notebook(text_list):


In [5]:
df_total['flair raw'] = flair_pos_sentences

In [39]:
nltk_pos_sentences = []
for text in tqdm_notebook(text_list):
    sentence = Sentence(text)
    tokens = []
    for token in sentence:
        tokens.append(token.__dict__['text'])
    tagged_sentence = nltk.pos_tag(tokens)
    nltk_pos_sentences.append(tagged_sentence)


<ipython-input-39-e686cea8fd54>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm_notebook(text_list):


In [41]:
df_total['nltk raw'] = nltk_pos_sentences

In [8]:
class HMMDecode:
    
    def __init__(self, model_path = "hmmmodel.txt"):
        
        self.model_path = model_path
        # variables to keep possible tags and words
        self.possible_tags = set()
        self.possible_words = set()
        
        # variable to keep possible tags count
        self.possible_tags_count = dict()
        
        # variables to keep the probabilities
        self.emission_probabilities = dict()
        self.transition_probabilities = dict()

        with open(self.model_path) as file:
            lines = file.readlines()
            for line in lines:
                var_array = line.split(" ")
                if var_array[0] == "em":
                    self.emission_probabilities[var_array[1]] = float(var_array[2].strip())
                    word_tag = var_array[1]
                    word = word_tag.rsplit("/")[0]
                    self.possible_words.add(word)
        
                elif var_array[0] == "tr":
                    self.transition_probabilities[var_array[1]] = float(var_array[2].strip())
        
                elif var_array[0] == "tg":
                    self.possible_tags.add(var_array[1].strip())
                    self.possible_tags_count[var_array[1]] = int(var_array[2].strip())
        return
        
    def smooth_probabilities(self, word, prev_tag, cur_tag):
        
        if (prev_tag + "/" + cur_tag) not in self.transition_probabilities:
            tr_prob = 1 / float(len(self.possible_tags) + self.possible_tags_count[prev_tag])
        else:
            tr_prob = self.transition_probabilities[prev_tag + "/" + cur_tag]
        
        if word not in self.possible_words:
            em_prob = 1
        elif (word + "/" + cur_tag) not in self.emission_probabilities:
            em_prob = 0
        else:
            em_prob = self.emission_probabilities[word + "/" + cur_tag]
            
        return em_prob, tr_prob
    
    def viterbi_algorithm(self, sentence):
        best_edge = dict()
        best_score = dict()
        words = sentence.split(" ")
        words = [word.strip() for word in words]
       
        for tag in self.possible_tags:
            em_prob, tr_prob = self.smooth_probabilities(words[0], "<s>", tag)
            best_score[(words[0], tag, 0)] = em_prob * tr_prob
            best_edge[(words[0], tag, 0)] = "<s>"

        for i in range(1, len(words)):
            for cur_tag in self.possible_tags:
                temp_score = 0
                if (words[i] in self.possible_words) and ((words[i] + "/" + cur_tag) not in self.emission_probabilities):
                    best_score[(words[i], cur_tag, i)] = temp_score
                else:
                    for prev_tag in self.possible_tags:
                        em_prob, tr_prob = self.smooth_probabilities(words[i], prev_tag, cur_tag)
                        score = best_score[(words[i-1], prev_tag, i-1)] * em_prob * tr_prob
                        best_score[(words[i], cur_tag, i)] = temp_score
                        
                        if score > temp_score:
                            temp_score = score
                            best_score[(words[i], cur_tag, i)] = score
                            best_edge[(words[i], cur_tag, i)] = prev_tag
        score = 0
        best_tag = None
        tagged_sentence = []
        nth_word = words[-1]
        words_length = len(words) - 1
        for tag in self.possible_tags:
            if best_score[(nth_word, tag, words_length)] > score:
                score = best_score[(nth_word, tag, words_length)]
                best_tag = tag
        tagged_sentence.append((nth_word, best_tag))
        
        for i in range(len(words) - 2, -1, -1):
            try:
                tagged_sentence.append((words[i], best_edge[(words[i+1], best_tag, i+1)]))
                best_tag = best_edge[(words[i+1], best_tag, i+1)]
            except:
                tagged_sentence.append((words[i], 'UNK'))
                best_tag = 'UNK'
        return tagged_sentence
    
    def tag_sentence(self, sentence, file_object):
        tagged_sentence = self.viterbi_algorithm(sentence)
        tagged_sentence = tagged_sentence[::-1]
        
        lnth = len(tagged_sentence)
        for i, word_tag in enumerate(tagged_sentence):
            word = word_tag[0]
            tag = word_tag[1]
            file_object.write(word + "/" + tag)
            if i != lnth - 1:
                file_object.write(" ")
        
        file_object.write("\n")
        
        return
    
    def run(self, infile, outfile):
        try:
            file_object = open(outfile, "w")
            with open(infile) as file:
                sentences = file.readlines()
                for i, sentence in enumerate(sentences):
                    self.tag_sentence(sentence, file_object)
            
            file_object.close()
            
        except Exception as e:
            print (e)
    
        return
    def predict(self, sentence):
        tagged_sentence = self.viterbi_algorithm(sentence)
        tagged_sentence = tagged_sentence[::-1]
        return tagged_sentence
    
model_path = './github/HMM-POS-Tagger/hmmmodel.txt'
hmm_decode_object = HMMDecode(model_path)



In [9]:
hmm_pos_sentences = []
for text in tqdm_notebook(text_list):
    tagged_sentence = hmm_decode_object.predict(text)
    hmm_pos_sentences.append(tagged_sentence)

<ipython-input-9-9a16f3fcf26a>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm_notebook(text_list):


In [10]:
df_total['hmm raw'] = hmm_pos_sentences

In [51]:
flair_raw = df_total['flair raw'].values.tolist()
nltk_raw = df_total['nltk raw'].values.tolist()
hmm_raw = df_total['hmm raw'].values.tolist()
df_total

,id,text,author,flair POS,nltk POS,hmm POS,flair raw,nltk raw,hmm raw
0,id26305,"This process, however, afforded me no means of...",EAP,"This <DT> process <NN> , <,> however <RB> , <,...","[(This, DT), (process, NN), (,, ,), (however, ...","[('This', 'DT'), ('process,', 'JJ'), ('however...","This <DT> process <NN> , <,> however <RB> , <,...","[(This, DT), (process, NN), (,, ,), (however, ...","[('This', 'DT'), ('process,', 'JJ'), ('however..."
1,id17569,It never once occurred to me that the fumbling...,HPL,It <PRP> never <RB> once <RB> occurred <VBD> t...,"[(It, PRP), (never, RB), (once, RB), (occurred...","[('It', 'PRP'), ('never', 'RB'), ('once', 'RB'...",It <PRP> never <RB> once <RB> occurred <VBD> t...,"[(It, PRP), (never, RB), (once, RB), (occurred...","[('It', 'PRP'), ('never', 'RB'), ('once', 'RB'..."
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,In <IN> his <PRP$> left <JJ> hand <NN> was <VB...,"[(In, IN), (his, PRP$), (left, JJ), (hand, NN)...","[('In', 'IN'), ('his', 'PRP$'), ('left', 'JJ')...",In <IN> his <PRP$> left <JJ> hand <NN> was <VB...,"[(In, IN), (his, PRP$), (left, JJ), (hand, NN)...","[('In', 'IN'), ('his', 'PRP$'), ('left', 'JJ')..."
3,id27763,How lovely is spring As we looked from Windsor...,MWS,How <WRB> lovely <JJ> is <VBZ> spring <NN> As ...,"[(How, WRB), (lovely, RB), (is, VBZ), (spring,...","[('How', 'WRB'), ('lovely', 'JJ'), ('is', 'VBZ...",How <WRB> lovely <JJ> is <VBZ> spring <NN> As ...,"[(How, WRB), (lovely, RB), (is, VBZ), (spring,...","[('How', 'WRB'), ('lovely', 'JJ'), ('is', 'VBZ..."
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,"Finding <VBG> nothing <NN> else <RB> , <,> not...","[(Finding, VBG), (nothing, NN), (else, RB), (,...","[('Finding', 'VBG'), ('nothing', 'NN'), ('else...","Finding <VBG> nothing <NN> else <RB> , <,> not...","[(Finding, VBG), (nothing, NN), (else, RB), (,...","[('Finding', 'VBG'), ('nothing', 'NN'), ('else..."
...,...,...,...,...,...,...,...,...,...
19574,id17718,"I could have fancied, while I looked at it, th...",EAP,"I <PRP> could <MD> have <VB> fancied <VBN> , <...","[(I, PRP), (could, MD), (have, VB), (fancied, ...","[('I', 'PRP'), ('could', 'MD'), ('have', 'VB')...","I <PRP> could <MD> have <VB> fancied <VBN> , <...","[(I, PRP), (could, MD), (have, VB), (fancied, ...","[('I', 'PRP'), ('could', 'MD'), ('have', 'VB')..."
19575,id08973,The lids clenched themselves together as if in...,EAP,The <DT> lids <NNS> clenched <VBD> themselves ...,"[(The, DT), (lids, NNS), (clenched, VBD), (the...","[('The', 'DT'), ('lids', 'NN'), ('clenched', '...",The <DT> lids <NNS> clenched <VBD> themselves ...,"[(The, DT), (lids, NNS), (clenched, VBD), (the...","[('The', 'DT'), ('lids', 'NN'), ('clenched', '..."
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP,Mais <FW> il <FW> faut <FW> agir <FW> that <DT...,"[(Mais, NNP), (il, NN), (faut, NN), (agir, NN)...","[('Mais', 'DT'), ('il', 'NN'), ('faut', '.'), ...",Mais <FW> il <FW> faut <FW> agir <FW> that <DT...,"[(Mais, NNP), (il, NN), (faut, NN), (agir, NN)...","[('Mais', 'DT'), ('il', 'NN'), ('faut', '.'), ..."
19577,id17513,"For an item of news like this, it strikes us i...",EAP,For <IN> an <DT> item <NN> of <IN> news <NN> l...,"[(For, IN), (an, DT), (item, NN), (of, IN), (n...","[('For', 'IN'), ('an', 'DT'), ('item', 'NN'), ...",For <IN> an <DT> item <NN> of <IN> news <NN> l...,"[(For, IN), (an, DT), (item, NN), (of, IN), (n...","[('For', 'IN'), ('an', 'DT'), ('item', 'NN'), ..."


In [55]:
re_tag = re.compile(r'\<(.*?)\>')
flair_POS = []
for string in flair_raw:
    tags = re_tag.findall(string)
    flair_POS.append(tags)

In [56]:
nltk_POS = []
for lst in nltk_raw:
    temp_lst = []
    for tup in lst:
        temp_lst.append(tup[1])
    nltk_POS.append(temp_lst)

In [61]:
hmm_POS = []
for lst in hmm_raw:
    temp_lst = []
    for tup in lst:
        temp_lst.append(tup[1])
    hmm_POS.append(temp_lst)

In [63]:
df_total['flair POS'] = flair_POS
df_total['nltk POS'] = nltk_POS
df_total['hmm POS'] = hmm_POS

In [64]:
df_total

,id,text,author,flair POS,nltk POS,hmm POS,flair raw,nltk raw,hmm raw
0,id26305,"This process, however, afforded me no means of...",EAP,"[DT, NN, ,, RB, ,, VBD, PRP, DT, NN, IN, VBG, ...","[DT, NN, ,, RB, ,, VBD, PRP, DT, NNS, IN, VBG,...","[DT, JJ, NN, IN, PRP, DT, NNS, IN, IN, DT, NN,...","This <DT> process <NN> , <,> however <RB> , <,...","[(This, DT), (process, NN), (,, ,), (however, ...","[('This', 'DT'), ('process,', 'JJ'), ('however..."
1,id17569,It never once occurred to me that the fumbling...,HPL,"[PRP, RB, RB, VBD, IN, PRP, IN, DT, NN, MD, VB...","[PRP, RB, RB, VBD, TO, PRP, IN, DT, NN, MD, VB...","[PRP, RB, RB, VBN, IN, PRP, IN, DT, NN, MD, VB...",It <PRP> never <RB> once <RB> occurred <VBD> t...,"[(It, PRP), (never, RB), (once, RB), (occurred...","[('It', 'PRP'), ('never', 'RB'), ('once', 'RB'..."
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[IN, PRP$, JJ, NN, VBD, DT, JJ, NN, NN, ,, IN,...","[IN, PRP$, JJ, NN, VBD, DT, JJ, NN, NN, ,, IN,...","[IN, PRP$, JJ, NN, VBD, DT, NN, ., '', IN, NN,...",In <IN> his <PRP$> left <JJ> hand <NN> was <VB...,"[(In, IN), (his, PRP$), (left, JJ), (hand, NN)...","[('In', 'IN'), ('his', 'PRP$'), ('left', 'JJ')..."
3,id27763,How lovely is spring As we looked from Windsor...,MWS,"[WRB, JJ, VBZ, NN, IN, PRP, VBD, IN, NNP, NNP,...","[WRB, RB, VBZ, JJ, IN, PRP, VBD, IN, NNP, NNP,...","[WRB, JJ, VBZ, NN, IN, PRP, VBD, IN, NNP, NNP,...",How <WRB> lovely <JJ> is <VBZ> spring <NN> As ...,"[(How, WRB), (lovely, RB), (is, VBZ), (spring,...","[('How', 'WRB'), ('lovely', 'JJ'), ('is', 'VBZ..."
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,"[VBG, NN, RB, ,, RB, RB, NN, ,, DT, NNP, VBD, ...","[VBG, NN, RB, ,, RB, RB, NN, ,, DT, NNP, VBD, ...","[VBG, NN, VBZ, RB, RB, IN, DT, NN, VBD, PRP$, ...","Finding <VBG> nothing <NN> else <RB> , <,> not...","[(Finding, VBG), (nothing, NN), (else, RB), (,...","[('Finding', 'VBG'), ('nothing', 'NN'), ('else..."
...,...,...,...,...,...,...,...,...,...
19574,id17718,"I could have fancied, while I looked at it, th...",EAP,"[PRP, MD, VB, VBN, ,, IN, PRP, VBD, IN, PRP, ,...","[PRP, MD, VB, VBN, ,, IN, PRP, VBD, IN, PRP, ,...","[PRP, MD, VB, VBN, IN, PRP, VBD, IN, NN, IN, D...","I <PRP> could <MD> have <VB> fancied <VBN> , <...","[(I, PRP), (could, MD), (have, VB), (fancied, ...","[('I', 'PRP'), ('could', 'MD'), ('have', 'VB')..."
19575,id08973,The lids clenched themselves together as if in...,EAP,"[DT, NNS, VBD, PRP, RB, IN, IN, IN, DT, NN, .]","[DT, NNS, VBD, PRP, RB, IN, IN, IN, DT, NN, .]","[DT, NN, IN, PRP, RB, RB, IN, IN, DT, NN]",The <DT> lids <NNS> clenched <VBD> themselves ...,"[(The, DT), (lids, NNS), (clenched, VBD), (the...","[('The', 'DT'), ('lids', 'NN'), ('clenched', '..."
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP,"[FW, FW, FW, FW, DT, VBZ, TO, VB, ,, DT, NNP, ...","[NNP, NN, NN, NN, WDT, VBZ, TO, VB, ,, DT, NNP...","[DT, NN, ., '', WDT, VBZ, TO, VB, DT, NN, RB, ...",Mais <FW> il <FW> faut <FW> agir <FW> that <DT...,"[(Mais, NNP), (il, NN), (faut, NN), (agir, NN)...","[('Mais', 'DT'), ('il', 'NN'), ('faut', '.'), ..."
19577,id17513,"For an item of news like this, it strikes us i...",EAP,"[IN, DT, NN, IN, NN, IN, DT, ,, PRP, VBZ, PRP,...","[IN, DT, NN, IN, NN, IN, DT, ,, PRP, VBZ, PRP,...","[IN, DT, NN, IN, NN, IN, IN, PRP, VBP, PRP, PR...",For <IN> an <DT> item <NN> of <IN> news <NN> l...,"[(For, IN), (an, DT), (item, NN), (of, IN), (n...","[('For', 'IN'), ('an', 'DT'), ('item', 'NN'), ..."


In [65]:
df_total.to_csv(path+'total_with_pos.csv') 

# Victorian Era Authorship Attribution

In [12]:
path = './dataset/Victorian Era Authorship Attribution/split/'

df_train = pd.read_csv(path+'train.csv', delimiter=',', index_col= 0, encoding='utf-8') 
df_test = pd.read_csv(path+'test.csv', delimiter=',', index_col= 0, encoding='utf-8') 
df_dev = pd.read_csv(path+'dev.csv', delimiter=',', index_col= 0, encoding='utf-8') 
df_total = pd.read_csv(path+'total.csv', delimiter=',', index_col= 0, encoding='utf-8') 


In [14]:
# id_list = df_total[''].values.tolist()
text_list = df_total['text'].values.tolist()
author_list = df_total['author'].values.tolist()

In [15]:
tagger = SequenceTagger.load('pos')
flair_pos_sentences = []
for text in tqdm_notebook(text_list):
    sentence = Sentence(text)    
    tagger.predict(sentence)
    tagged_sentence = sentence.to_tagged_string()
    flair_pos_sentences.append(tagged_sentence)

2020-10-22 10:15:31,994 loading file /home/joey/.flair/models/en-pos-ontonotes-v0.5.pt


<ipython-input-15-5ef82793719b>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm_notebook(text_list):


KeyboardInterrupt: 

In [ ]:
df_total['flair POS'] = flair_pos_sentences

In [ ]:
nltk_pos_sentences = []
for text in tqdm_notebook(text_list):
    sentence = Sentence(text)
    tokens = []
    for token in sentence:
        tokens.append(token.__dict__['text'])
    tagged_sentence = nltk.pos_tag(tokens)
    nltk_pos_sentences.append(tagged_sentence)

In [ ]:
df_total['nltk POS'] = nltk_pos_sentences

In [ ]:
model_path = './github/HMM-POS-Tagger/hmmmodel.txt'
hmm_decode_object = HMMDecode(model_path)

In [ ]:
hmm_pos_sentences = []
for text in tqdm_notebook(text_list):
    tagged_sentence = hmm_decode_object.predict(text)
    hmm_pos_sentences.append(tagged_sentence)

In [ ]:
df_total['hmm POS'] = hmm_pos_sentences

In [ ]:
df_total.to_csv(path+'total_with_pos.csv')

In [ ]:
# path = './dataset/Spooky Author Identification/split/'

# df_train = pd.read_csv(path+'train.csv', delimiter=',', index_col= 0, encoding='utf-8') 
# df_test = pd.read_csv(path+'test.csv', delimiter=',', index_col= 0, encoding='utf-8') 
# df_dev = pd.read_csv(path+'dev.csv', delimiter=',', index_col= 0, encoding='utf-8') 
# df_total = pd.read_csv(path+'total.csv', delimiter=',', index_col= 0, encoding='utf-8') 

# df_train

# id_list = df_total['id'].values.tolist()
# text_list = df_total['text'].values.tolist()
# author_list = df_total['author'].values.tolist()




# for text in text_list:
#     sentence = Sentence(text_list[9])    
#     tagger.predict(sentence)
#     tagged_sentence = sentence.to_tagged_string()
    
#     break



